# Knowledge base System
- 문서를 이용한 벡터DB 구성 실습

In [1]:
!pip install docx2txt

# 문서 로딩

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax.docx')
docu = loader.load()

# 문서 분할

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)
splitter

In [3]:
doc_list = loader.load_and_split(text_splitter=splitter)

In [4]:
len(doc_list)

183

# 임베딩

In [6]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 DB 적재 - chromaDB
- 인메모리 DB -> restart하면 날아감

In [ ]:
!pip install langchain-chroma

In [7]:
from langchain_chroma import Chroma
database = Chroma.from_documents(  # 새로 생성
    documents=doc_list,
    embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma'
)

In [8]:
from langchain_chroma import Chroma
database = Chroma(   # 기존 저장본 불러오기
    collection_name='chroma-tax',
    persist_directory='./chroma',
    embedding_function=embedding
)

# Retrieval

In [9]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)

In [9]:
retrieved_docs

[Document(id='222a6d77-c35e-4b52-9f75-5b79061630fa', metadata={'source': './tax.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외

# augmentation

In [12]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [11]:
prompt = f"""
[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [context]내용만을 참조하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
"""

In [12]:
response = llm.invoke(prompt)

In [13]:
print(response.content)

연봉 5천만원인 직장인의 소득세는 아래의 세율을 적용하여 계산합니다.

1. **연봉 5천만원에 대한 과세표준**: 5천만원은 1,400만원 초과 ~ 5,000만원 이하 구간에 해당합니다.

2. **세율 적용**:
   - 기본 세액: 84만원 (1,400만원 초과 구간에 대한 기본 금액)
   - 추가 세액: 5,000만원 초과 금액 (즉, 5,000만원 - 1,400만원 = 3,600만원)에 대해 15% 적용.

계산식은 다음과 같습니다:
- 기본 세액: 84만원
- 추가 세액: 3,600만원 × 15% = 540만원

따라서 총 소득세는:
- 총 소득세 = 84만원 + 540만원 = 624만원

결과적으로, 연봉 5천만원인 직장인의 소득세는 **624만원**입니다.


# 질의의 정규화

In [14]:
# 질의의 품질을 높이기 위한 정규화 과정 추가
def normalize_query(q: str) -> str:
    # 숫자: '5천만원' → '5,000만원'
    q = q.replace("오천만원","5,000만원").replace("5천만원","5,000만원")
    # 용어: 연봉 → (과세) 과세표준 후보어 추가
    q = q.replace("연봉", "과세표준")
    q = q.replace("직장인", "거주자")
    # 의도 신호: 계산 키워드 보강
    if "계산" not in q:
        q = q.rstrip("요?").rstrip("?") + " 계산 기준과 누진공제를 적용해 계산"
    return q
query_n = normalize_query(query)

In [15]:
retrieved_docs = database.similarity_search(query_n, k=5)
retrieved_docs

[Document(id='9b6cd7f3-5fb6-42da-87d3-764d782e5697', metadata={'source': './tax.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율(계산식)\n\n1,400만원 이하\n\n과세표준의 6%\n\n1,400만원 초과 ~ 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15%)\n\n5,000만원 초과 ~ 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24%)\n\n8,800만원 초과 ~ 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35%)\n\n1억5천만원 초과 ~ 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38%)\n\n3억원 초과 ~ 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40%)\n\n5억원 초과 ~ 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42%)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45%)\n\n의 45%)"\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009

# query 정규화, 고도화 작업이 필요한 이유

In [16]:
query, query_n

('연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 '과세표준 5,000만원인 거주자의 소득세는 얼마인가 계산 기준과 누진공제를 적용해 계산')

# Retrieval와 RetrievalQA 체인

In [21]:
retriever = database.as_retriever()  # 기본 리트리버
retriever.invoke(query_n)

[Document(id='9b6cd7f3-5fb6-42da-87d3-764d782e5697', metadata={'source': './tax.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율(계산식)\n\n1,400만원 이하\n\n과세표준의 6%\n\n1,400만원 초과 ~ 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15%)\n\n5,000만원 초과 ~ 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24%)\n\n8,800만원 초과 ~ 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35%)\n\n1억5천만원 초과 ~ 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38%)\n\n3억원 초과 ~ 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40%)\n\n5억원 초과 ~ 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42%)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45%)\n\n의 45%)"\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009

In [22]:
from langchain import hub
prompt = hub.pull('rlm/rag-prompt')  # 범용 래그 프롬프트

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [23]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
from langchain.chains import RetrievalQA
# 소득세법 문서 추출 체인
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={'prompt':prompt}
)

In [27]:
response = qa_chain.invoke({'query':query_n})

In [28]:
response

{'query': '과세표준 5,000만원인 거주자의 소득세는 얼마인가 계산 기준과 누진공제를 적용해 계산',
 'result': '과세표준이 5,000만원인 경우, 소득세는 84만원 + (1,400만원을 초과하는 금액인 3,600만원에 15%)를 적용하여 계산합니다. 따라서, 소득세는 84만원 + 540만원 = 624만원입니다. 요약하자면, 해당 거주자의 소득세는 624만원입니다.'}

# Retrieval를 위한 키워드 사전 (용어 사전)

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ['사람을 나타내는 표현 --> 거주자']

promtpt_dic = ChatPromptTemplate.from_template(
    f"""
    사용자의 질문을 보고, 아래 사전을 참고하여 사용자의 질문을 변경하세요.\n
    만약 변경할 필요가 없다고 판단되면, 사용자의 질문을 변경하지 않아도 됩니다.\n
    그런 경우에는 질문만 리턴하세요.
    
    사전 : {dictionary}
    
    질문 : {{question}}
    """
)

In [35]:
dictionary_chain = promtpt_dic | llm | StrOutputParser()
tax_chain = {'query':dictionary_chain} | qa_chain

In [32]:
query

'연봉 5천만원인 직장인의 소득세는 얼마인가요?'

In [33]:
new_query = dictionary_chain.invoke({'question':query})
new_query

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [36]:
tax_chain.invoke({'question':query})

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 84만원 + (1,400만원을 초과하는 금액의 15%)로 계산됩니다. 따라서, 5천만원의 경우 초과분인 3,600만원에 15%를 적용하면 540만원이 추가되어 소득세는 총 624만원이 됩니다.'}

# 해보기

In [37]:
query1 = '회사 사정으로 9월 급여를 못 줬습니다. 연말이 지나면 세금은 어떤 기준으로 처리하나요?'

In [38]:
dictionary = [
    "직원, 사원, 근로자, 직장인 --> 근로소득자",
    "월급, 급여, 임금, 페이 --> 근로소득",
    "세금 떼다, 세금공제, 원천공제, 세금 --> 원천징수",
    "보너스, 성과급, 인센티브--> 상여",
    "연말 --> 12월 31일",
]

promtpt_dic = ChatPromptTemplate.from_template(
    f"""
    사용자의 질문을 보고, 아래 사전을 참고하여 사용자의 질문을 변경하세요.\n
    만약 변경할 필요가 없다고 판단되면, 사용자의 질문을 변경하지 않아도 됩니다.\n
    그런 경우에는 질문만 리턴하세요.
    
    사전 : {dictionary}
    
    질문 : {{question}}
    """
)

In [39]:
dictionary_chain = promtpt_dic | llm | StrOutputParser()
tax_chain = {'query':dictionary_chain} | qa_chain

In [40]:
new_query1 = dictionary_chain.invoke({'question':query1})
new_query1

'회사 사정으로 9월 근로소득을 못 줬습니다. 12월 31일이 지나면 원천징수는 어떤 기준으로 처리하나요?'

In [45]:
retrieved_docs = database.similarity_search(new_query1, k=5)
retrieved_docs

[Document(id='c451b1c2-81a8-4016-8e74-4edd0d0d6eed', metadata={'source': './tax.docx'}, page_content='[본조신설 2009. 12. 31.]\n\n\n\n제3관 근로소득에 대한 원천징수 <개정 2009. 12. 31.>\n\n\n\n제134조(근로소득에 대한 원천징수시기 및 방법) ① 원천징수의무자가 매월분의 근로소득을 지급할 때에는 근로소득 간이세액표에 따라 소득세를 원천징수한다.\n\n② 원천징수의무자는 다음 각 호의 어느 하나에 해당할 때에는 제137조, 제137조의2 또는 제138조에 따라 소득세를 원천징수하며, 제1호의 경우 다음 연도 2월분의 근로소득에 대해서는 제1항에서 규정하는 바에 따라 소득세를 원천징수한다.<개정 2010. 12. 27.>\n\n1. 해당 과세기간의 다음 연도 2월분 근로소득을 지급할 때(2월분의 근로소득을 2월 말일까지 지급하지 아니하거나 2월분의 근로소득이 없는 경우에는 2월 말일로 한다. 이하 같다)\n\n2. 퇴직자가 퇴직하는 달의 근로소득을 지급할 때\n\n③ 원천징수의무자가 일용근로자의 근로소득을 지급할 때에는 그 근로소득에 근로소득공제를 적용한 금액에 원천징수세율을 적용하여 계산한 산출세액에서 근로소득세액공제를 적용한 소득세를 원천징수한다.\n\n④ 삭제<2010. 12. 27.>\n\n⑤ 근로소득자의 근무지가 변경됨에 따라 월급여액(月給與額)이 같은 고용주에 의하여 분할지급되는 경우의 소득세는 변경된 근무지에서 그 월급여액 전액에 대하여 제1항부터 제3항까지의 규정을 적용하여 원천징수하여야 한다.<개정 2010. 12. 27.>\n\n[전문개정 2009. 12. 31.]\n\n[제목개정 2010. 12. 27.]\n\n\n\n제135조(근로소득 원천징수시기에 대한 특례) ① 근로소득을 지급하여야 할 원천징수의무자가 1월부터 11월까지의 근로소득을 해당 과세기간의 12월 31일까지 지급하지 아니한 경우에는 그 근로소득을 12월 31일에 

In [43]:
tax_chain.invoke({'question':query1})

{'query': '회사 사정으로 9월 근로소득을 못 줬습니다. 12월 31일이 지나면 원천징수는 어떤 기준으로 처리하나요?',
 'result': '9월 근로소득을 지급하지 못한 경우, 해당 소득은 12월 31일에 지급한 것으로 간주되어 원천징수됩니다. 따라서, 원천징수의무자는 12월 31일에 소득세를 원천징수해야 합니다. 이 규정은 2010년 12월 27일 개정된 내용을 기반으로 합니다.'}